In [1]:
import os
import fitz  # PyMuPDF
import cv2
import numpy as np
import pandas as pd
from paddleocr import PaddleOCR
from tqdm import tqdm
import re
import unicodedata

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/paddle/utils/cpp_extension/extension_utils.py:711: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)


In [46]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# initialize Paddle
ocr = PaddleOCR(use_angle_cls=True, lang='en',show_log=False)

def extract_banners_from_pdf(pdf_path):
    super_headers = []

    doc = fitz.open(pdf_path)
    for page in doc:
        # Render page -> image
        pix = page.get_pixmap(dpi=200)
        img = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.h, pix.w, pix.n)
        if pix.n == 4:
            img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)
        cnts, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for cnt in cnts:
            x,y,w,h = cv2.boundingRect(cnt)
            if w < 200 or h < 20:
                continue
            crop = img[y:y+h, x:x+w]
            black_ratio = cv2.countNonZero(thresh[y:y+h, x:x+w]) / float(w*h)
            if black_ratio < 0.6:
                continue

            inv = cv2.bitwise_not(crop)
            try:
                result = ocr.ocr(inv, cls=True)
            except Exception:
                continue
            if not result:
                continue

            # Safely extract all recognized text lines
            lines = []
            for block in result:
                if not block:
                    continue
                for line in block:
                    # line is typically [ [x1,y1], [text, conf], ... ]
                    if len(line) >= 2 and isinstance(line[1], (list, tuple)):
                        lines.append(line[1][0])
            text = " ".join(lines).strip()
            if not text:
                continue

            # classify banner size
            if h > 50:
                super_headers.append(text)

    # dedupe while preserving order
    uniq_super = list(dict.fromkeys(super_headers))

    return uniq_super

def scan_folder_and_save_csv(root_folder : str, out_headers : str, check_folders : int = None):
    all_super = []
    
    if check_folders is not None:
        list_items = list(set(os.listdir(root_folder)) - set(['.DS_Store']))[0:check_folders]
    else:
        list_items = list(set(os.listdir(root_folder)) - set(['.DS_Store']))

    for case in tqdm(sorted(list_items), desc="Folders"):
        case_path = os.path.join(root_folder, case)
        if not os.path.isdir(case_path):
            continue
        # find the application PDF
        excludes = {'acknowledgement', 'withdrawal', 'bvb', 'rdv', 'passport','summary'}
        pdfs = [f for f in os.listdir(case_path) if (name := f.lower()).endswith(".pdf")
                and "application" in name and not any(ex in name for ex in excludes)]
        if not pdfs:
            continue
        pdf_path = os.path.join(case_path, pdfs[0])
        sh = extract_banners_from_pdf(pdf_path)
        all_super.extend(sh)
    
    # dedupe
    uniq_super = sorted(set(all_super))
    
    # save
    pd.Series(uniq_super, name="super_header").to_csv(out_headers, index=False)
    
    print(f"Saved {len(uniq_super)} super‐headers to {out_headers}")
    return uniq_super

In [4]:
def slugify(text: str) -> str:
    t = unicodedata.normalize("NFKD", text)
    t = "".join(c for c in t if not unicodedata.combining(c))
    t = t.lower()
    return re.sub(r"[^a-z0-9]+", "_", t).strip("_")

def initials(text: str) -> str:
    return "".join(w[0] for w in text.split() if w).lower()

def process_applications(root_folder: str, subheaders_list: list[str], check_folders : int = None):
    sub_set = {s.lower() for s in subheaders_list}

    rows = []
    if check_folders is not None: 
        list_items = list(set(os.listdir(root_folder)) - set(['.DS_Store']))[0:check_folders]
    else:
        list_items = list(set(os.listdir(root_folder)) - set(['.DS_Store']))

    for case in tqdm(sorted(list_items), desc="Cases"):
        case_path = os.path.join(root_folder, case)
        if not os.path.isdir(case_path):
            continue

        # find application PDF
        app_pdf = None
        excludes = {'acknowledgement', 'withdrawal', 'bvb', 'passport', 'summary'}
        for fn in os.listdir(case_path):
            low = fn.lower()
            if ( "application" in low and low.endswith(".pdf")  and not any(ex in low for ex in excludes)):
                app_pdf = os.path.join(case_path, fn)
                break
        if not app_pdf:
            continue

        # collect all spans
        doc = fitz.open(app_pdf)
        spans = []
        for page in doc:
            for block in page.get_text("dict")["blocks"]:
                if block["type"] != 0:
                    continue
                for line in block["lines"]:
                    for span in line["spans"]:
                        font = span.get("font", span.get("fontname", "")).lower()
                        spans.append({
                            "text": span["text"].strip(),
                            "size": round(span["size"], 1),
                            "flags": span["flags"],
                            "font": font,
                        })

        # counters
        ofm_count = 0
        cv_count = 0
        ehd_count = 0

        # count occurrences of Employment history details anywhere
        for s in spans:
            if s["text"].strip().lower() == "employment history details":
                ehd_count += 1

        current_sup = None
        current_sub = None
        rec = {"sample_id": case}

        for i, s in enumerate(spans):
            t = s["text"]
            if not t:
                continue

            bold = bool(s["flags"] & 2) or ("bold" in s["font"])
            low = t.lower()

            # super-header?  size=14.0 flags=16 bold font
            if bold and s["size"] == 14.0 and s["flags"] == 16:
                current_sup = t
                current_sub = None
                if low == "other family members":
                    ofm_count += 1
                if low == "country visited":
                    cv_count += 1
                continue

            # sub-header?
            if bold and low in sub_set:
                current_sub = t
                continue

            # field/question?
            is_field = (not bold) and (t.endswith("?") or ":" in t)
            if is_field:
                # skip if the question looks like page footer or layout artifact
                if any(kw in t.lower() for kw in ("generated", "reference number", "trn")):
                    continue
                
                # sanitize parentheses
                key_txt = re.sub(r"\(.*?\)", "X", t)
                base = key_txt.split(":", 1)[0].rstrip("?").strip()
                col_slug = slugify(base)
                parts = [col_slug]
                if current_sup:
                    parts.append(initials(current_sup))
                if current_sub:
                    parts.append(initials(current_sub))
                colname = "_".join(parts)

                # find next bold span as valid answer
                ans = ""
                for nxt in spans[i + 1:]:
                    nxt_bold = bool(nxt["flags"] & 2) or ("bold" in nxt["font"])
                    val = nxt["text"].strip()
                    if nxt_bold and val and val.lower() not in ("personal privacy", "official: sensitive"):
                        ans = val
                        break
                rec[colname] = ans
                continue

            # special: Value in Australian dollars
            if "value in australian dollars" in low:
                val = ""
                for nxt in spans[i + 1:]:
                    nxt_bold = bool(nxt["flags"] & 2) or ("bold" in nxt["font"])
                    if nxt_bold and re.match(r"^[\d,\.]+$", nxt["text"].replace(" ", "")):
                        val = nxt["text"].strip()
                        break
                if val:
                    rec["value_in_australian_dollars"] = val
                continue
        
        # add generation date from first page
        generated_value = None
        for span in spans[:100]:  # assuming ~100 spans on first page is enough
            txt = span["text"]
            if txt.lower().startswith("generated:"):
                generated_value = txt.split(":", 1)[-1].strip()
                break
        rec["generated"] = generated_value
        
        # store counts
        rec["other_family_members_count"] = ofm_count
        rec["country_visited_count"] = cv_count
        rec["employment_history_details_count"] = ehd_count

        rows.append(rec)

    df = pd.DataFrame(rows).set_index("sample_id")
    return df

In [5]:
df_grant_list = pd.read_csv('header_Grant.csv')
df_ref_list = pd.read_csv('header_Refusal.csv')

headers_list = list(set(df_grant_list.super_header.tolist() + df_ref_list.super_header.tolist()))

In [6]:
type_case = 'Grant'
#df_grant_list = pd.read_csv('header_Grant.csv')
#list_headers_g = df_grant_list.super_header.tolist()
##list_headers_g = scan_folder_and_save_csv('data_all/Info Students/'+type_case, 'header_'+type_case+'.csv', check_folders = 500)
df_g = process_applications('data_all/Info Students/'+type_case,headers_list, check_folders = None)

type_case = 'Refusal'
#df_ref_list = pd.read_csv('header_Refusal.csv')
#list_headers_r = df_ref_list.super_header.tolist()
#list_headers_r = scan_folder_and_save_csv('data_all/Info Students/'+type_case, 'header_'+type_case+'.csv',check_folders = 300)
df_r = process_applications('data_all/Info Students/'+type_case,headers_list, check_folders = None)

Cases:  58%|█████▊    | 5382/9315 [17:54<13:20,  4.92it/s]  

MuPDF error: syntax error: cannot load shading function (2233 0 R)

MuPDF error: syntax error: cannot load shading function (2233 0 R)

MuPDF error: syntax error: cannot load shading function (2233 0 R)

MuPDF error: syntax error: cannot load shading function (2233 0 R)

MuPDF error: syntax error: cannot load shading function (2233 0 R)

MuPDF error: syntax error: cannot load shading function (2233 0 R)

MuPDF error: syntax error: cannot load shading function (2233 0 R)

MuPDF error: syntax error: cannot load shading function (2233 0 R)

MuPDF error: syntax error: cannot load shading function (2233 0 R)

MuPDF error: syntax error: cannot load shading function (2233 0 R)

MuPDF error: syntax error: cannot load shading function (2233 0 R)

MuPDF error: syntax error: cannot load shading function (2233 0 R)

MuPDF error: syntax error: cannot load shading function (2233 0 R)

MuPDF error: syntax error: cannot load shading function (2233 0 R)

MuPDF error: syntax error: cannot load shading f

Cases: 100%|██████████| 740/740 [03:46<00:00,  3.27it/s]


In [7]:
df_g['type_case'] = 'granted'
df_r['type_case'] = 'refused'

In [8]:
df_cases = pd.concat([df_g,df_r])

In [12]:
df_cases.to_csv('data_onevisa.csv', index=False)

In [13]:
df_nans = pd.DataFrame(df_cases.isna().sum()).reset_index()
df_nans.columns = ['cols_n','numbers']

In [14]:
df_nans = df_nans.sort_values(by='numbers')
df_nans.shape

(4662, 2)

In [15]:
df_nans.tail()

,cols_n,numbers
2994,account_holder_s_address_raf,7335
2995,signature_raf_ouo,7335
2996,comments_raf_ouo,7335
2990,account_no_raf,7335
4661,date_to_e_ud,7335


In [16]:
#delete_cols = df_nans[df_nans.numbers>=1600].cols_n.unique().tolist()

In [17]:
cleaned_df = df_cases.dropna(how='all')

cleaned_df.shape

(7336, 4662)

In [18]:
cleaned_df.type_case.value_counts()

type_case
granted    6741
refused     595
Name: count, dtype: int64

In [19]:
# [i for i in cleaned_df.columns.tolist() if 'e_ehd' in i]

In [20]:
# pd.DataFrame(cleaned_df.loc['Yaritza Solange Carvajal Perez'])

In [21]:
remove_index = cleaned_df[(cleaned_df['other_family_members_count']==0) & (cleaned_df['country_visited_count']==0)  & (cleaned_df['employment_history_details_count']==0)][['type_case']].index

In [22]:
cleaned_df_2 = cleaned_df.drop(index=remove_index.tolist())

In [23]:
cleaned_df_2 = cleaned_df_2[cleaned_df_2.current_location_ac_cl.notnull()]

In [24]:
# TBA???
cleaned_df_2.shape

(6742, 4662)

In [25]:
df_nans = pd.DataFrame(cleaned_df_2.isna().sum()).reset_index()
df_nans.columns = ['cols_n','numbers']
df_nans = df_nans.sort_values(by='numbers')
df_nans.shape

(4662, 2)

In [26]:
delete_cols = df_nans[df_nans.numbers>=cleaned_df_2.shape[0]].cols_n.unique().tolist()
cleaned_df_3 = cleaned_df_2.drop(columns=delete_cols+[''])
cleaned_df_3.shape

(6742, 649)

In [27]:
df_nans = pd.DataFrame(cleaned_df_3.isna().sum()).reset_index()
df_nans.columns = ['cols_n','numbers']
df_nans = df_nans.sort_values(by='numbers')
df_nans[df_nans.numbers>1].head(10)

,cols_n,numbers
45,passport_number_cdc,2
76,accompanying_member_of_the_family_unit_in_this...,2
73,mobile_cell_phone_ar_ctn,2
72,business_phone_ar_ctn,2
71,postal_code_ar_pa,2
70,state_or_province_ar_pa,2
69,suburb_town_ar_pa,2
68,address_ar_pa,2
40,family_name_cdc,2
41,given_names_cdc,2


In [28]:
cleaned_df_3[cleaned_df_3.intelligence_agency_x_cd.isna()][['intelligence_agency_x_cd','type_case']]

,intelligence_agency_x_cd,type_case
sample_id,,
Aitor Laiglesia Beamonde,NaN,granted
Alejandro Collado Olucha,NaN,granted
Alicia Madurell Maña,NaN,granted
Benji Marino Jaimes Jerez,NaN,granted
Daniel Gil Malpartida,NaN,granted
Daniel Zardain Salcedo,NaN,granted
Eric Matas Benavente,NaN,granted
Ignacio Rouco Forcada,NaN,granted
Jose Manuel Bezanilla Corte,NaN,granted


In [29]:
from collections import defaultdict
import re

def print_exclusive_per_row_columns(df):
    groups = defaultdict(list)

    # Agrupa por prefijo sin sufijo (_cd, _od, _dvod, etc.)
    for col in df.columns:
        base = re.sub(r'_[a-z]{2,5}$', '', col)
        groups[base].append(col)

    for base, cols in groups.items():
        if len(cols) < 2:
            continue

        sub_df = df[cols]
        
        # Para cada fila, contar cuántas columnas no son NaN
        non_nan_counts_per_row = sub_df.notna().sum(axis=1)

        # Si en todas las filas hay solo una columna con valor no NaN
        if (non_nan_counts_per_row == 1).all():
            print(f"{base}: {cols}")


In [30]:
print_exclusive_per_row_columns(cleaned_df_3)

home_phone_cd: ['home_phone_cd_ctn', 'home_phone_cd_ctnia']
business_phone_cd: ['business_phone_cd_ctn', 'business_phone_cd_ctnia']
mobile_cell_phone_cd: ['mobile_cell_phone_cd_ctn', 'mobile_cell_phone_cd_ctnia']
in_australia: ['in_australia_hd', 'in_australia_vtoc']
now_removed_from_official_records: ['now_removed_from_official_records_cd', 'now_removed_from_official_records_od']
has_any_applicant_ever_been_the_subject_of_an_arrest_warrant_or_interpol_notice: ['has_any_applicant_ever_been_the_subject_of_an_arrest_warrant_or_interpol_notice_cd', 'has_any_applicant_ever_been_the_subject_of_an_arrest_warrant_or_interpol_notice_od', 'has_any_applicant_ever_been_the_subject_of_an_arrest_warrant_or_interpol_notice_dvod']
where_no_conviction_was_recorded: ['where_no_conviction_was_recorded_cd', 'where_no_conviction_was_recorded_od', 'where_no_conviction_was_recorded_dvod']
has_any_applicant_ever_been_named_on_a_sex_offender_register: ['has_any_applicant_ever_been_named_on_a_sex_offender_regi

In [31]:
# cleaned_df_3[['authority_in_australia_cd', 'authority_in_australia_domt', 'authority_in_australia_doms', 'authority_in_australia_od', 'authority_in_australia_dvod']]

In [32]:
from collections import defaultdict
import re

def unify_exclusive_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    groups = defaultdict(list)

    # Agrupar por base eliminando el sufijo (2–5 letras tras último guion bajo)
    for col in df.columns:
        base = re.sub(r'_[a-z]{2,5}$', '', col)
        groups[base].append(col)

    for base, cols in groups.items():
        if len(cols) < 2:
            continue

        sub_df = df[cols]

        # Check if only one non-NaN value per row
        if (sub_df.notna().sum(axis=1) == 1).all():
            # Create unified column with suffix _x
            unified_col = base + "_x"
            df[unified_col] = sub_df.bfill(axis=1).iloc[:, 0]

            # Drop original columns
            df.drop(columns=cols, inplace=True)

            print(f"Unified: {cols} -> {unified_col}")

    return df


cleaned_df_4 = unify_exclusive_columns(cleaned_df_3)

Unified: ['home_phone_cd_ctn', 'home_phone_cd_ctnia'] -> home_phone_cd_x
Unified: ['business_phone_cd_ctn', 'business_phone_cd_ctnia'] -> business_phone_cd_x
Unified: ['mobile_cell_phone_cd_ctn', 'mobile_cell_phone_cd_ctnia'] -> mobile_cell_phone_cd_x
Unified: ['in_australia_hd', 'in_australia_vtoc'] -> in_australia_x
Unified: ['now_removed_from_official_records_cd', 'now_removed_from_official_records_od'] -> now_removed_from_official_records_x
Unified: ['has_any_applicant_ever_been_the_subject_of_an_arrest_warrant_or_interpol_notice_cd', 'has_any_applicant_ever_been_the_subject_of_an_arrest_warrant_or_interpol_notice_od', 'has_any_applicant_ever_been_the_subject_of_an_arrest_warrant_or_interpol_notice_dvod'] -> has_any_applicant_ever_been_the_subject_of_an_arrest_warrant_or_interpol_notice_x
Unified: ['where_no_conviction_was_recorded_cd', 'where_no_conviction_was_recorded_od', 'where_no_conviction_was_recorded_dvod'] -> where_no_conviction_was_recorded_x
Unified: ['has_any_applicant_

In [33]:
cleaned_df_4.shape

(6742, 603)

In [34]:
df_nans = pd.DataFrame(cleaned_df_4.isna().sum()).reset_index()
df_nans.columns = ['cols_n','numbers']
df_nans = df_nans.sort_values(by='numbers')
df_nans[df_nans.numbers<=10].shape

(92, 2)

In [35]:
cleaned_df_4.to_csv('data_onevisa_postprocess.csv', index=True)

In [36]:
cleaned_df_4.shape

(6742, 603)

In [37]:
cleaned_df_4.type_case.value_counts()

type_case
granted    6174
refused     568
Name: count, dtype: int64

In [299]:
summary = pd.DataFrame({
        "number_unique": cleaned_df_4.nunique(),
        "number_nan": cleaned_df_4.isna().sum(),
        "dtype": cleaned_df_4.dtypes
    })

In [300]:
summary[summary.number_unique>1400]

,number_unique,number_nan,dtype
coe_code_coed,1549,51,object
family_name_pa_pd,1603,0,object
date_of_birth_pa_pd,1411,0,object
passport_number_pa_pd,1628,0,object
family_name_nic,1542,57,object
identification_number_nic,1567,57,object
family_name_cdc,1603,0,object
date_of_birth_cdc,1411,0,object
passport_number_cdc,1628,0,object
address_cd_ra,1518,107,object


In [301]:
summary.dtype.value_counts()

dtype
object    449
int64       3
Name: count, dtype: int64

In [302]:
summary.sort_values(by='number_unique').head()

,number_unique,number_nan,dtype
description_of_the_offence_od,1,1627,object
list_countries_on/s_c,1,1627,object
other_itwobaticc,1,1627,object
order_of_a_tribunal_or_court_or_other_similar_authority_for_the_personal_protection_of_another_person_od,1,1621,object
country_cd_cor,1,1521,object


In [303]:
summary[(summary.number_unique<=200) & (summary.number_nan<=200)].shape

(106, 3)